<a href="https://colab.research.google.com/gist/ashepherd/55de702cac1d69583a198447c1288709/fqm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib
!pip install owlrl
!pip install pyshacl

In [ ]:
import rdflib
from rdflib import Graph, URIRef, Literal, Namespace, BNode
from rdflib.namespace import RDF, RDFS, SH
import pyshacl
import owlrl

In [ ]:

# Create an RDF graph
g = Graph()

# Parse the ontology into the graph
g.parse("FAIRness-quality-measures-ontology.ttl")

# https://www.bobdc.com/blog/cmdlineowl/
# Do the inferencing. See
# https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.DeductiveClosure.html#owlrl.DeductiveClosure
# for other owlrl.* choices.
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(g)

FQM_PREFIX = "http://w3id.org/fairness-quality-measures/"
FQM = Namespace(FQM_PREFIX)


In [ ]:
query = """
    PREFIX fqm: <http://w3id.org/fairness-quality-measures/>

    SELECT DISTINCT ?indicator ?inspecting_class ?principle ?dimension_rank ?principle_rank ?indicator_level ?category_class
    WHERE {
        ?indicator rdf:type/rdfs:subClassOf fqm:FAIRQualityMeasure .
        ?indicator fqm:forPrinciple ?principle .
        ?principle fqm:rank ?principle_rank .
        ?principle fqm:hasDimension/fqm:rank ?dimension_rank .
        ?indicator fqm:inspectsCategoryClass ?category_class .
        ?indicator fqm:indicatorNumber ?indicator_level .
        ?inspecting_class fqm:satisfiesFAIRQualityMeasure ?indicator .
    }
    ORDER BY ?dimension_rank ?principle_rank ?indicator_level ?category_class
"""

results = g.query(query)
print(results)
if len(results) > 0:
  shapes_graph = Graph()
  shapes_graph.bind("fqm", FQM)
  shapes_graph.bind("sh", SH)
  shapes_graph.bind("rdf", RDF)
  shapes_graph.bind("rdfs", RDFS)

  for row in results:
    print(f"{row}")
    class_existence_shape = BNode()
    shapes_graph.add((class_existence_shape, RDF.type, SH.NodeShape))
    shapes_graph.add((class_existence_shape, RDFS.label, Literal(str(row.inspecting_class).replace(FQM_PREFIX, ""))))
    shapes_graph.add((class_existence_shape, SH.targetNode, row.inspecting_class))  # We're targeting ex:ABC

    print(f"{row.dimension_rank}-{row.principle_rank}-{row.indicator_level}-{row.category_class} => {row}")
    #print(f"{row}")

  shapes_graph.serialize(destination="shapes-graph.ttl", format="turtle")

(rdflib.term.URIRef('http://w3id.org/fairness-quality-measures/CoreIndicator_F1-01D'), rdflib.term.URIRef('http://w3id.org/fairness-quality-measures/DataAssignedIdentifier'), rdflib.term.URIRef('https://w3id.org/fair/principles/terms/F1'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.URIRef('http://w3id.org/fairness-quality-measures/Data'))
1-1-1-http://w3id.org/fairness-quality-measures/Data => (rdflib.term.URIRef('http://w3id.org/fairness-quality-measures/CoreIndicator_F1-01D'), rdflib.term.URIRef('http://w3id.org/fairness-quality-measures/DataAssignedIdentifier'), rdflib.term.URIRef('https://w3id.org/fair/principles/terms/F1'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))